In [1]:
from matplotlib import pyplot as plt
import numpy as np
import torch as tr
from torch import nn
import pandas as pd
from torch.utils.tensorboard import SummaryWriter
import csv
import draw_rna.draw as draw
from draw_rna.draw_utils import seq2col
import os 
import csv

In [2]:
secdist = pd.read_csv('RNA_dist/data/SequenceIdentity_f_all.csv')
names=secdist['id']
aux=np.array(secdist.iloc[:,1:])
aux2=secdist.columns[1:]
secdist = pd.DataFrame(aux,index=aux2,columns=aux2)

In [3]:
medioides = pd.read_csv('RNA_dist/medioide.csv')

813

In [5]:
def file_opener(file_name):
    data_set=[]
    with open(file_name, newline = '') as data:                                                                                         
        data_reader = csv.reader(data, delimiter=',')
        for dat in data_reader:
            data_set.append(dat)
    return data_set
data_set=file_opener('RNA_dist/data/archiveII_220808.csv')

### utils

In [7]:
def seq_to_pair(seq):
    seq_np=list(seq)
    seq_open=[]
    pairs=[]
    for it,char in enumerate(seq_np):
        if (char=='('):
            seq_open.append(it+1)
        if (char==')'):
            pairs.append([seq_open[-1],it+1])
            seq_open=seq_open[:-1]
    pairs=pairs[::-1]
    # pairs=', '.join(str(e) for e in pairs)
    # pairs='['+pairs+']'
    return pairs


def replace_E(seq):
    str=list(seq)
    for it in range(len(str)):
        print(str[it])
        if(str[it]=='('):
            break    
        else:
            str[it]='-'
    print('SALIO')
    for it in range(len(str)-1,0,-1):
        if(str[it]==')'):
            break   
        else:
            str[it]='-'
    return str
    
    
def dot_to_motif(seq, struct):
    """Get motifs from dot-bracket notation structure. Requieres bpRNA.pl script"""

    with open('tmp.dbn', 'w') as fout:
        fout.write('>id\n')
        fout.write(f'{seq}\n')
        fout.write(f'{struct}\n')
        
    os.system('perl extras/bpRNA.pl tmp.dbn')
    with open('tmp.st') as fin:
        k = 0
        for line in fin:
            if line[0] == '#':
                continue
            if k == 2:
                motif = line.strip()
                break
            else:
                k += 1
    os.remove('tmp.dbn')
    os.remove('tmp.st')
    return motif

def seq_len(seq):
   lenght=len(list(seq))
   return lenght

def make_data_set(data_set):
    data=[['id','sequence','structure','base_pairs','len']]
    for i in data_set[1:]:
        if seq_len(i[3]) < 800:
            data.append([i[1],i[3],i[4],seq_to_pair(i[4]),seq_len(i[3])])
      

        # data=data+i[0]+',SOLUTION NMR,'+i[1]+','+i[2]+','+'"'+pair+'"'+','+str(lenght)+'\n'
    writer=pd.DataFrame(data)
    writer.to_csv('dist03.csv',sep=',',header=False,index=False)
    return data

def valid_mask(seq):
    """Create a NxN mask with valid canonic pairings."""
    valid_pairs = [{"G", "C"}, {"A", "U"}, {"G", "U"}]
    mask = tr.zeros((len(seq), len(seq)), dtype=tr.float32)
    for i in range(len(seq)):
        for j in range(len(seq)):
            if i != j:
                if {seq[i], seq[j]} in valid_pairs:
                    mask[i, j] = 1
                    mask[j, i] = 1
    return mask




In [8]:
data=make_data_set(data_set)

# Dataset

In [9]:
import torch as tr
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import json
from torch.nn.functional import cross_entropy

In [10]:
class OneHot():
    def __init__(self):
        self.pad_token = "-"
        self.vocabulary = ['A', 'U', 'G', 'C'] 
        self.emb_size = 4
        
    def seq2onehot(self, seq):
        emb = tr.zeros((self.emb_size, len(seq)), dtype=tr.float)
        for k, nt in enumerate(seq):
            # print(f'letra = {nt} indice = {k}')
            if nt == self.pad_token:
                continue
            if nt in self.vocabulary:
                emb[self.vocabulary.index(nt), k] = 1
            else:
                emb[:, k] = 1/self.emb_size
            # print(f'embedding = {emb}')
        return emb     

In [11]:
class Data_dist(Dataset):

    def __init__(self,dataset_path, min_len=0, max_len=200,padding=True,random_pad=False):
        dat=pd.read_csv(dataset_path,index_col=None)
        dat = dat[(dat.len > min_len) & (dat.len < max_len)]
        self.lenght=dat.len
        self.max_len=max_len
        self.id=dat.id
        self.seq=dat.sequence
        self.struct=dat.structure
        self.pair=[json.loads(dat.base_pairs.iloc[i]) for i in range(len(dat))]
        self.n_samples=dat.shape[0]
        self.padding = padding
        self.pad_token = '-'
        self.random_pad = random_pad
        self.embedding = OneHot()

    def __getitem__(self,index):
        id = self.id.iloc[index]
        len_seq = len(self.seq.iloc[index])
        connections = tr.tensor(self.pair[index]) -1
        sequence,Mc = self.pad_sequence(self.seq.iloc[index],connections)
        seq_emb=self.embedding.seq2onehot(sequence)
        mask = valid_mask(sequence)
        return seq_emb,Mc,len_seq,mask,id
    def __len__(self): 
        return self.n_samples

        
    def pad_sequence(self, sequence, connections):
        """Pad sequence and create connection matrix to max length"""
        left_pad, right_pad = 0, 0 
        if self.padding:
            left_pad = 0
            right_pad = self.max_len - len(sequence)
            if self.random_pad:
                left_pad = np.random.randint(right_pad)
                right_pad -= left_pad


        sequence = self.pad_token*left_pad + sequence + self.pad_token*right_pad
        # Conection matrix: -1 padding, 0 is not connected, 1 is connected
        Mc = -tr.ones((len(sequence), len(sequence)), dtype=tr.float32)      
        if right_pad > 0:
            Mc[left_pad:-right_pad, left_pad:-right_pad] = 0
        else:
            Mc[left_pad:, left_pad:] = 0

        if len(connections)>0:
            # for i in range(len(connections)):
            #     Mc[connections[][0] + left_pad, connections[i][1] + left_pad] = 1
            Mc[connections[:,0] + left_pad, connections[:,1] + left_pad] = 1
            Mc[connections[:,1] + left_pad, connections[:,0] + left_pad] = 1
        # else:
            # print(f"Warning: sequence {sequence} do not have any connections")
        return sequence, Mc.long()

# Metricas

In [13]:
from sklearn.metrics import f1_score
#se suele usar f1 score para todo lo que sean matrices?????????!
def get_f1(ref_batch,predict_batch,L,th=0.5):
    f1 = []
    for ref,predict,l in zip(ref_batch,predict_batch,L):
        ind = tr.where(ref!=-1)
        ref = ref[ind].view(l, l)
        predict = predict[ind].view(l, l)

        #esto si no lo entiendo
        predict = tr.sigmoid(predict) > th

        #parte para tener la matriz triangular superior
        ind = tr.triu_indices(ref.shape[0], ref.shape[1], offset=1)
        ref = ref[ind[0], ind[1]].numpy().ravel()
        predict = predict[ind[0], ind[1]].numpy().ravel()
        f1.append(f1_score(ref, predict, zero_division=1))
    return tr.tensor(f1).mean()

In [21]:
def cluster_score(id,medioides):
    offset_fulldat = tr.empty(len(id),len(medioides))
    cluster_offset=tr.empty(len(medioides))
    
    for i in range(len(id)):
        id_pos=np.where(np.array(secdist['id'])==id[i])[0][0]
        for j in range(len(medioides)):
            cluster_offset[j]=(secdist.iloc[id_pos][medioides.iloc[j][0]])
        offset_fulldat[i]=cluster_offset
    return offset_fulldat

def concatenate_data (offset_list,sec_data,device):
    return tr.cat((tr.flatten(sec_data,start_dim=1),offset_list.clone().detach().to(device)),dim=1).to(device)

# Modelo

In [20]:
class ignacioDist(nn.Module):
    def __init__(self, embedding_dim=4, device="cpu", negative_weight=.1, lr=1e-3, 
    logger=None, pred_l1=.01, use_scheduler=False,offset=30,max_len=200, **kwargs):
        super().__init__()
        self.device=device
        self.k = offset
        self.len=max_len
        # self.rank = rank
        #por que espesificamente el adam
        #para luego escalar de 0 a 1 el crossentropy
        self.class_weight = tr.tensor([negative_weight, 1.]).float().to(device)
        self.build_graph(embedding_dim,**kwargs)
        self.optim =  tr.optim.Adam(self.parameters(), lr=lr)

        self.to(device)


    
    def build_graph (self,emb_dim,kernel=9,filters=50,rank=32):
        pad = (kernel-1)//2
        dilation = 4
        self.cnn = nn.Sequential(nn.Conv1d(in_channels=emb_dim, out_channels=filters, 
                                        kernel_size=kernel, padding = pad, stride=1),
                                nn.ReLU(),
                                nn.BatchNorm1d(filters),
                                nn.Conv1d(in_channels=filters, out_channels=filters, 
                                        kernel_size=kernel, dilation = dilation,
                                        padding=dilation*pad, stride=1),
                                nn.ReLU(),
                                nn.BatchNorm1d(filters),
                                nn.Conv1d(in_channels=filters, out_channels=filters, 
                                        kernel_size=kernel, dilation = dilation,
                                        padding=dilation*pad, stride=1),
                                nn.ReLU(),
                                nn.BatchNorm1d(filters),
                                nn.Conv1d(in_channels=filters, out_channels=filters//2, 
                                        kernel_size=kernel, padding = pad, stride=1),
                                nn.Sigmoid(),)  
        
        self.convsal1 = nn.Conv1d(in_channels=filters//2, out_channels=rank, kernel_size=kernel, 
                                  padding=pad, stride=1)
        self.convsal2 = nn.Conv1d(in_channels=filters//2,  out_channels=rank, kernel_size=kernel,
                                  padding = pad, stride=1)
        self.linear1 = nn.Sequential(nn.Flatten(),
                                    nn.Linear(in_features=(self.len*rank)+self.k, out_features=(self.len*rank),bias=False))
        self.linear2 = nn.Sequential(nn.Flatten(),
                                    nn.Linear(in_features=(self.len*rank)+self.k, out_features=(self.len*rank),bias=False))
        
    def forward (self,x,id_list):
        start = time.time()
        n = x.shape[2]
        y = self.cnn(x)

        offset_list = cluster_score(id_list,medioides)
    
        ya = self.convsal1(y)
        
        yb = self.convsal2(y)
        
        
        ya = concatenate_data(offset_list,ya,self.device)
        yb = concatenate_data(offset_list,yb,self.device)


        ya = self.linear1(ya)
        yb = self.linear2(yb)
        
        ya = ya.view(y.shape[0],32,n)
        yb = yb.view(y.shape[0],32,n)
        
        ya = tr.transpose(ya, -1, -2)

        y = ya @ yb

        yt =  tr.transpose(y, -1, -2)
        
        y = (y+yt)/2
        # aca concatenas la informacion de cada cluster
        # print('longitud de y antes de la concatenacion ',len(y),y.shape[1],y.shape[2])
        
        # print('despues de la concatenacion ',len(y))
        # print('pasa la capa linear',y.shape[0],y.shape[1])
        
        #por que pone vectores adentro de otro vectores
        y = y.view(-1, n, n)
        return y
#loss and optim
    def loss_func(self,yt,y):

        #devuelta lo mismo
        y = y.view(y.shape[0], -1)
        yt = yt.view(yt.shape[0], -1)

        #no es la diferencia?
        pred_l1_loss =  tr.mean(tr.abs(yt[y!=-1])) * 0.01 


        #por que no usar devuelta view
        yt = yt.unsqueeze(1)
        #hizo algo el codigo anterior desp de esto? tipo no se deshizo?
        yt = tr.cat((-yt, yt), dim=1) 
        error = cross_entropy(yt, y, ignore_index=-1, weight=self.class_weight)
        loss= error + pred_l1_loss
        return loss
    
    def tr(self,data_loader):
        self.train()
        avg_loss=0
        avg_f1 = 0
        train_step = 0
        for batch in data_loader:
            x = batch[0].to(self.device)
            y = batch[1].to(self.device)
            id_z = batch[4]

            self.optim.zero_grad()  
            y_tr = self.forward(x,id_z)
        
            loss = self.loss_func(y_tr,y)
            f1 = get_f1(y.cpu(), y_tr.detach().cpu(), batch[2])
            avg_f1 += f1
            avg_loss += loss
            loss.backward() 
            self.optim.step()
            train_step+=1

        avg_f1 /= len(data_loader)
        avg_loss /= len(data_loader)
        return avg_loss,avg_f1
    def tst(self,data_loader):
        self.eval() 
        avg_loss=0
        avg_f1=0
        test_step = 0
        #desactivamos el gradiente por que no hacemos backward()
        with tr.no_grad():       
            for batch in data_loader:
                x = batch[0].to(self.device)
                y = batch[1].to(self.device)
                id_z = batch[4]
                #por que aca y no desp
                y_tst = self.forward(x,id_z)
                # y_tst=tr.tensor([[x], *batch[2:]])
                loss = self.loss_func(y_tst,y)
                #.detach()?????????
                f1 = get_f1(y.cpu(), y_tst.detach().cpu(), batch[2])
                avg_loss += loss
                avg_f1 += f1
                test_step+=1
            avg_loss /= len(data_loader)
            avg_f1 /= len(data_loader)
            
            return avg_loss,avg_f1


In [30]:
import time
start = time.time()
for i in range(100):
    print('lala')
end = time.time()
print((end-start)*1000)


lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
lala
0.7467269897460938


# main


In [22]:
import torch as tr
from torch.utils.data import DataLoader, random_split
maxlen=512
out_path = f"results/"
batch_size = 8
DEVICE = "cuda" 
model = ignacioDist(kernel=11, device=DEVICE,max_len=maxlen) 
secdist = pd.read_csv('RNA_dist/data/SequenceIdentity_f_all.csv')
# model.load_state_dict(tr.load(f"{out_path}model_200len.pmt"))
print('entra a hacer las particiones')
dataset=Data_dist('dist03.csv',max_len=maxlen)


n_train = int(0.8 * len(dataset))
n_test = int(0.1 * len(dataset))
n_val = len(dataset) - n_train - n_test



train_data, val_data, test_data = random_split(dataset, [n_train, n_val, n_test], generator=tr.Generator().manual_seed(42))
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

print("train batches", len(train_loader))
print("val batches", len(val_loader))
print("test batches", len(test_loader))
best_f1, patience_counter = 0, 0
for epoch in range(150):
    train_loss,train_f1 = model.tr(train_loader)
    print(f'train loop epoch {epoch}')
    val_loss,val_f1 = model.tst(val_loader)
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_epoch = epoch
        tr.save(model.state_dict(), f"{out_path}model_130len_fc2.pmt")
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter>50:
            break
    print(f"epoch {epoch}: train loss {train_loss:.2f} val loss {val_loss:.2f} train f1 {train_f1:.2f} val f1 {val_f1:.2f}")
print("Start test")
model.load_state_dict(tr.load(f"{out_path}model_130len_fc2.pmt"))
test_loss, test_f1 = model.tst(test_loader)
print(f"best epoch {best_epoch} best_valf1 {best_f1:.3f} test_loss {test_loss:.3f} test_f1 {test_f1:.3f}")

entra a hacer las particiones
train batches 387
val batches 49
test batches 49
train loop epoch 0
epoch 0: train loss 2.98 val loss 0.09 train f1 0.11 val f1 0.20


KeyboardInterrupt: 

                                                                                                                                        nvidia-smi -l 5

In [ ]:
print("Start test")
model.load_state_dict(tr.load(f"{out_path}model_130len_fc.pmt"))
test_loss, test_f1 = model.tst(test_loader)
print(f"best epoch {best_epoch} best_valf1 {best_f1:.3f} test_loss {test_loss:.3f} test_f1 {test_f1:.3f}")

Start test
best epoch 28 best_valf1 0.926 test_loss 0.049 test_f1 0.899
